In [ ]:
!pip install geopandas
!pip install contextily
!pip install eli5
!pip install pdpbox
!pip install shap

In [ ]:
import pandas as pd
from shapely.geometry import  Point
import geopandas as gdp
import contextily as ctx
# import geoplot as gplt
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import cm
import urllib.request
import shutil
import zipfile
import os
import re

import lightgbm as lgb
import eli5
from eli5.sklearn import PermutationImportance
from lightgbm import LGBMClassifier
from pdpbox import pdp,get_dataset,info_plots
import shap

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') # 此處需要登入google帳號
# 獲取授權碼之後輸入即可連動雲端硬碟
train = pd.read_csv("/content/gdrive/My Drive/train.csv")
test = pd.read_csv("/content/gdrive/My Drive/test.csv")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
train.drop_duplicates(inplace = True)
train.replace({"X":-120.5,"Y":90.0},np.NaN,inplace=True)

Simp = SimpleImputer(strategy = "mean")
for district in train['PdDistrict'].unique():
    train.loc[train['PdDistrict']==district,["X","Y"]] = Simp.fit_transform(train.loc[train['PdDistrict']==district,["X","Y"]])
    test.loc[test['PdDistrict']==district,["X","Y"]] = Simp.fit_transform(test.loc[test['PdDistrict']==district,["X","Y"]])
    

In [ ]:
def feature_engineering(data):
    # data['Date'] = pd.to_datetime(data['Dates'].date)
    data['n_days'] = (
        data['Date'] - data['Date'].min()).apply(lambda x: x.days)
    data['Day'] = data['Dates'].dt.day
    data['DayOfWeek'] = data['Dates'].dt.weekday
    data['Month'] = data['Dates'].dt.month
    data['Year'] = data['Dates'].dt.year
    data['Hour'] = data['Dates'].dt.hour
    data['Minute'] = data['Dates'].dt.minute
    data['Block'] = data['Address'].str.contains('block', case=False)
    
    data.drop(columns=['Dates','Date','Address'], inplace=True)
        
    return data

In [ ]:
train = feature_engineering(train)
train.drop(columns=['Descript','Resolution'], inplace=True)
test = feature_engineering(test)
train.head()

AttributeError: ignored

In [ ]:
le1 = LabelEncoder()
train['PdDistrict']=le1.fit_transform(train['PdDistrict'])
test['PdDistrict'] = le1.transform(test['PdDistrict'])
# 使用另一個變數就要另一個全新的Encoder
le2 = LabelEncoder()
y=le2.fit_transform(train.pop('Category'))

train_X,val_X,train_y,val_y=train_test_split(train,y)

model = LGBMClassifier(objective = 'multiclass',num_class=39).fit(train_X,train_y)

perm = PermutationImportance(model).fit(train_X,train_y)
eli5.show_weights(perm,feature_names=val_X.columns.tolist())